In [6]:
import torch
from functools import partial


# SGD

## use SGD to estimate quadratic equation

We are trying to estimate a quadratic equation. To do so, we initialize random parameters of the equation. We use mean square error as loss function to calculate the gradient. Then we use SGD to optimize the parameters.

In [7]:
# mean square error
def mse(preds, acts):
    return ((preds - acts) ** 2).mean()


def quad(a, b, c, x):
    return a * x**2 + b * x + x


def mk_quad(a, b, c):
    return partial(quad, a, b, c)


# target model
f = mk_quad(2, 3, 4)
f(2)

# assume some data points
x = torch.linspace(-2, 2, 20)[:, None]
torch.manual_seed(42)

# Generate a tensor of random numbers with the same shape as f(x)
# torch.rand_like(f(x)) generates random numbers between 0 and 1
# with the same shape as f(x). We scale and shift it to the desired range.
random_numbers = torch.rand_like(f(x)) * 10 - 5

# dataset
y = f(x) + random_numbers


# loss function
def quad_mse(params):
    f = mk_quad(*params)
    return mse(f(x), y)


# initial params
params = torch.tensor([4, 5.0, 7.0])
params.requires_grad_()

loss = quad_mse(params)
loss

loss.backward()
params.grad

tensor([11.2822,  6.9424,  0.0000])

Let's calculate the the SGD and loss manually. 

Here is the loss function:
$$
\text{mse}(f(x), y) = \frac{1}{n} \sum_{i=1}^n (f(x_i) - y_i)^2
$$

where $n$ is the number of data points.

To calculate the gradient of the loss, we start with a generic expression

$$
f(x_i) = a x_i^2 + b x_i + c
$$

then

$$
\frac{\partial f(x_i)}{\partial a}
= x_i^2.
$$

Then we have the loss function, where $y_i$ are $y$ value in $(x_i, y_i)$ in the training data and $f(x_i)$ is the target $y$ value

$$
L = \frac{1}{n} \sum_{i=1}^n (f(x_i) - y_i)^2.
$$

So for parameter $a$, our gradient is:

$$
\frac{\partial L}{\partial a}
= \frac{2}{n} \sum_{i=1}^n (f(x_i) - y_i) \cdot x_i^2.
$$

The process is similar for $b$ and $c$.

To perform SGD, we subtract $grad \times learning rate$ from the weight:

$params \mathrel{-}= learning\,rate \times grad$


In [8]:
print("target params", f)
print("initial params", params)
print("initial values of x: ", x[:2])

target params functools.partial(<function quad at 0x7fe06004c9a0>, 2, 3, 4)
initial params tensor([4., 5., 7.], requires_grad=True)
initial values of x:  tensor([[-2.0000],
        [-1.7895]])


In [9]:
lr = 0.23
params = torch.tensor([4, 5.0, 7.0])
params.requires_grad_()
for _ in range(10):
    loss = quad_mse(params)
    print("loss ", loss.item())
    loss.backward()
    params.data -= lr * params.grad.data
    params.grad = None

loss  23.300430297851562
loss  12.930697441101074
loss  10.261430740356445
loss  8.984521865844727
loss  8.224483489990234
loss  7.7517900466918945
loss  7.455584526062012
loss  7.269739627838135
loss  7.153112888336182
loss  7.079920768737793


In [10]:
params

tensor([2.6931, 2.6446, 7.0000], requires_grad=True)

## Use SGD to train image classification on the MNIST dataset

### dataset exploration

In [11]:
#!wget "https://s3.amazonaws.com/fast-ai-sample/mnist_tiny.tgz" -O "data/mnist_tiny.tar.gz" && tar -xzf "data/mnist_tiny.tar.gz" -C data/

In [12]:
from pathlib import Path

path = Path("data/mnist_tiny/")
print(list(path.iterdir()))
threes = sorted((path / "train" / "3").iterdir())
sevens = sorted((path / "train" / "7").iterdir())
threes[1]

[PosixPath('data/mnist_tiny/labels.csv'), PosixPath('data/mnist_tiny/test'), PosixPath('data/mnist_tiny/valid'), PosixPath('data/mnist_tiny/train'), PosixPath('data/mnist_tiny/models')]


PosixPath('data/mnist_tiny/train/3/7030.png')

In [13]:
from PIL import Image

im3_path = threes[2]
im3 = Image.open(im3_path)
im3

In [14]:
from numpy import array

im3_array = array(im3)
im3_array[15:25, 4:10]

array([[  0,   0,   0,   0,   0,   0],
       [ 16,  42,   0,   0,   0,   0],
       [197, 102,   0,   0,   0,   0],
       [254, 166,   5,   0,   0,   0],
       [212, 254, 193,  77,  30,   0],
       [ 29, 194, 254, 254, 242, 213],
       [  0,  20, 165, 254, 254, 254],
       [  0,   0,   3,  12,  65, 149],
       [  0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0]], dtype=uint8)

In [15]:
import torch

im3_t = torch.as_tensor(im3_array)
im3_t[15:25, 4:10]


tensor([[  0,   0,   0,   0,   0,   0],
        [ 16,  42,   0,   0,   0,   0],
        [197, 102,   0,   0,   0,   0],
        [254, 166,   5,   0,   0,   0],
        [212, 254, 193,  77,  30,   0],
        [ 29, 194, 254, 254, 242, 213],
        [  0,  20, 165, 254, 254, 254],
        [  0,   0,   3,  12,  65, 149],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0]], dtype=torch.uint8)

In [16]:
import pandas as pd

df = pd.DataFrame(im3_t)
df.style.set_properties(**{"font-size": "6pt"}).background_gradient("Greys")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,11,130,224,255,254,244,95,2,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,3,128,243,254,239,229,231,254,254,103,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,92,254,254,212,52,0,5,107,254,223,11,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,28,212,196,54,0,0,0,16,229,254,24,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,16,5,0,0,0,0,0,206,254,24,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,250,254,24,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,166,254,215,9,0,0,0,0,0,0,0


### Prediction using averages
First, we use the most simple classification method, we calculate an "ideal" images of 3s and 7s with averages, then we use a loss function to calculate test images' distance with the 3 and 8, and classify the image based on the lower loss score.

In [17]:
# convert images to tensors arrays
seven_tensors = [torch.as_tensor(array(Image.open(o))) for o in sevens]
three_tensors = [torch.as_tensor(array(Image.open(o))) for o in threes]

# convert tensor array to individual tensors
stacked_sevens = torch.stack(seven_tensors).float() / 255
stacked_threes = torch.stack(three_tensors).float() / 255
stacked_threes.shape


torch.Size([346, 28, 28])

we will create our ideal 3 and 7

In [18]:
mean3 = stacked_threes.mean(0)
mean7 = stacked_sevens.mean(0)

In [19]:
import torchvision.transforms as T


def tensor_image(t):
    return T.ToPILImage()((t * 255).byte())


# Convert tensor to PIL Image (scale values to 0-255 and convert to uint8)
mean3_pil = tensor_image(mean3)
mean3_pil

let's pick a test image

In [20]:
sample = stacked_threes[32]
tensor_image(sample)

we will calculate mean absolute error (L1 norm) the mean square error (L2 norm)

In [21]:
dist_3_abs = (sample - mean3).abs().mean()
dist_3_sqr = ((sample - mean3) ** 2).mean().sqrt()
dist_3_abs, dist_3_sqr


(tensor(0.1350), tensor(0.2499))

In [22]:
dist_7_abs = (sample - mean7).abs().mean()
dist_7_sqr = ((sample - mean7) ** 2).mean().sqrt()
dist_7_abs, dist_7_sqr


(tensor(0.1233), tensor(0.2432))

In [23]:
# use built in loss functions
(
    torch.nn.functional.l1_loss(sample.float(), mean3),
    torch.nn.functional.mse_loss(sample, mean3),
)

(tensor(0.1350), tensor(0.0625))

In [24]:
print("Is the sample a 3? ", dist_3_sqr > dist_7_sqr)

Is the sample a 3?  tensor(True)


Next, we will will apply our classification on the validation set

In [25]:
valid_threes = sorted((path / "valid" / "3").iterdir())
valid_sevens = sorted((path / "valid" / "7").iterdir())
valid_seven_tensors = [torch.as_tensor(array(Image.open(o))) for o in valid_sevens]
valid_three_tensors = [torch.as_tensor(array(Image.open(o))) for o in valid_threes]

# convert tensor array to individual tensors
valid_sevens_stacked = torch.stack(valid_seven_tensors).float() / 255
valid_three_stacked = torch.stack(valid_three_tensors).float() / 255


In [26]:
valid_three_stacked.shape, valid_sevens_stacked.shape

(torch.Size([346, 28, 28]), torch.Size([353, 28, 28]))

Define our loss function with l1 loss

In [27]:
# loss function
def mnist_l1_loss(a, b):
    return (a - b).abs().mean((-1, -2))


mnist_l1_loss(sample, mean3)

tensor(0.1350)

In [28]:
valid_three_distance = mnist_l1_loss(valid_three_stacked, mean3)

valid_three_distance.shape

torch.Size([346])

In [29]:
# leverage broadcast to calculate the distance between samples and the idea 3
(valid_three_stacked - mean3).shape

torch.Size([346, 28, 28])

classify each sample in the validation set, calculate the accuracy

In [30]:
def is_3(x):
    return mnist_l1_loss(x, mean3) < mnist_l1_loss(x, mean7)


accuracy_3s = is_3(valid_three_stacked).float().mean()
accuracy_7s = (1 - is_3(valid_sevens_stacked).float()).mean()

accuracy_3s, accuracy_7s, (accuracy_3s + accuracy_7s) / 2


(tensor(0.9335), tensor(0.9972), tensor(0.9653))

we can see that the accuracy is very high in both cases.

### Train a linear model for classification

Now we use a linear model as the classification model.

### Train an NN for classification